In [1]:
import pandas as pd
from lyricsgenius import Genius
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import requests
import json
import concurrent.futures
import time
from langdetect import detect

## Get liked songs using Spotify API

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Set up authentication with the correct scope
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id = "a69702265fff4ff4be13fb46ee91c46f",
    client_secret = '793809bcafa34fe594dad746fc34d7c6',
    redirect_uri="http://localhost:5555/callback",  # Your Redirect URI
    scope="user-library-read"  # Scope to access your liked songs
))

# Function to get all liked songs and return them as a DataFrame
def get_liked_songs_df():
    songs = []
    limit = 50  # Max number of items per API call
    offset = 0   # Start from the first song

    while True:
        # Get a page of saved tracks
        results = sp.current_user_saved_tracks(limit=limit, offset=offset)
        tracks = results["items"]

        if not tracks:
            break  # No more tracks left

        for item in tracks:
            track = item["track"]
            song_name = track["name"]
            artist_name = track["artists"][0]["name"]
            songs.append([song_name, artist_name])

        offset += limit  # Move to the next batch of 50 songs

    # Create a DataFrame from the list of songs
    df = pd.DataFrame(songs, columns=["Song", "Artist"])
    return df

# Fetch and display the liked songs DataFrame
df_liked_songs = get_liked_songs_df()
df_liked_songs

,Song,Artist
0,Don't Worry Bout Me,Boot Juice
1,I Don't Need Nothin,Boot Juice
2,Take Me There,Boot Juice
3,Falling,HAIM
4,i like the way you kiss me - BBC Radio 1 Live ...,Cat Burns
...,...,...
1319,Rescue,Lauren Daigle
1320,Wild,Beach House
1321,You Came To Me,Beach House
1322,Chariot,Beach House


## Genius

In [17]:

token = "bAZcBjviMr_-Mz7dBvxO6vnfwqGqaLg9Uyex4Rb62rjDgL5vdUszvABmxBe3-aI9"
import time
from lyricsgenius import Genius
from requests.exceptions import Timeout

genius = Genius(token, timeout=10)  # Set a longer timeout
lyrics = []
genius.remove_section_headers = True

for index, row in df_liked_songs.iterrows():
    try:
        artist = genius.search_artist(row['Artist'], max_songs=1, allow_name_change=True)
        if artist is None:
            lyrics.append("none")
        else:
            song = artist.song(row['Song'])
            if song is None:
                lyrics.append("none")
            else:
                lyrics.append(song.lyrics)
    except Timeout:
        lyrics.append("none")
        print(f"Timeout error at index {index} ({row['Artist']} - {row['Song']}), skipping.")
    
    time.sleep(1)  # Avoid hitting rate limits




Searching for songs by Boot Juice...

Changing artist name to 'DJ JUICY BOOTY'
Song 1: "$WAGGANOMIC$"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Don't Worry Bout Me" by DJ JUICY BOOTY...
No results found for: 'Don't Worry Bout Me DJ JUICY BOOTY'
Searching for songs by Boot Juice...

Changing artist name to 'DJ JUICY BOOTY'
Song 1: "$WAGGANOMIC$"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "I Don't Need Nothin" by DJ JUICY BOOTY...
No results found for: 'I Don't Need Nothin DJ JUICY BOOTY'
Searching for songs by Boot Juice...

Changing artist name to 'DJ JUICY BOOTY'
Song 1: "$WAGGANOMIC$"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Take Me There" by DJ JUICY BOOTY...
No results found for: 'Take Me There DJ JUICY BOOTY'
Searching for songs by HAIM...

Song 1: "The Wire"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Falling" by HAIM...
Done.
Searching for songs by

KeyboardInterrupt: 

In [12]:
len(lyrics)

542

In [15]:
lyrics[0]

"I lost my best friend\nI never had him\nHe's disappearing off a ledge\nI guess he left the way he came in\nMade out of peppermint\nAll white and red pins\nPins and needles on my tongue\nIt's not exactly what I wanted\nI'm holding you on my tongue\nPoured into my cup\nIt's something you whipped up\nFrom the skins of fruits and flowers\nI've forgotten all the names of\nAnd on the next day\nI woke up with brand new legs\nI still don't know what it was\nBut I remember how it tasted\nI'm holding you on my tongue\nYou like your shirts with\nShort collars and small prints\nAnd I like walking home so drunk\nI can't remember what the day is\nGod, you're so quiet (God, you're so quiet)\nSo brave and so kind (So brave and so kind)\nAnd If you keep talking like that\nThen I might walk home with you all night\nI'm holding you on my tongue\nI'm holding you on my tongue"

In [182]:
lyrics[-1]

"Oh my Otis won't you come back down\nI'm on my way but I need a little time\nShook my soul and you saved my life\nI'm on my way but I need a little time\n\nI need some love from a Georgia man\nBang it out in Dawson\nI get him then I lost him\nPart time preacher in the traveling band\nThat sugar on his shoulder\n\nOh my Otis won't you come back down\nI'm on my way but I need a little time\nShook my soul and you saved my life\nI'm on my way but I need a little time\n\nSing me up then break my heart\nThose Cigarettes and Coffee\nLook at the man he's got me\nBig Ol' Ranch keeps us apart\nAnd that Tramp from California\n\nOh my Otis won't you come back down\nI'm on my way but I need a little time\nShook my soul and you saved my life\nI'm on my way but I need a little time\n\nSugar in the morning you gotta get on up\nListen to the sound and you gotta get on up\n\nOh my Otis won't you come back down\nI'm on my way but I need a little time\nShook my soul and you saved my life\nI'm on my way b

In [222]:
next_df = df_liked_songs[782:]
next_df

,Song,Artist,Lyrics
782,And I Love Her,Kurt Cobain,None
783,Why Are Sundays So Depressing,The Strokes,None
784,Today - 2011 Remaster,The Smashing Pumpkins,None
785,All Apologies,Nirvana,None
786,Perfect,The Smashing Pumpkins,None
...,...,...,...
1312,Rescue,Lauren Daigle,None
1313,Wild,Beach House,None
1314,You Came To Me,Beach House,None
1315,Chariot,Beach House,None


In [223]:
next_df['lyrics'] = lyrics

/var/folders/3m/chgnvz6n0lvb48zy37bx_p6c0000gn/T/ipykernel_33234/1185623660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  next_df['lyrics'] = lyrics


In [3]:
df_w_lyrics = df_w_lyrics.append(next_df)
df_w_lyrics


NameError: name 'df_w_lyrics' is not defined

In [225]:
df_w_lyrics.to_csv("liked_songs.csv")

## Adding artist followers and song duration

In [233]:
import requests
import pandas as pd
import time  # To prevent rate limits

# Replace with your Spotify API credentials
CLIENT_ID = "a69702265fff4ff4be13fb46ee91c46f"
CLIENT_SECRET = '793809bcafa34fe594dad746fc34d7c6'


# Get Spotify API Token
def get_access_token():
    url = "https://accounts.spotify.com/api/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {"grant_type": "client_credentials"}
    
    response = requests.post(url, headers=headers, data=data, auth=(CLIENT_ID, CLIENT_SECRET))
    response_json = response.json()
    
    return response_json.get("access_token")

# Fetch track details (ID, duration, artist ID)
def get_track_details(song, artist, token):
    search_url = f"https://api.spotify.com/v1/search?q=track:{song} artist:{artist}&type=track&limit=1"
    headers = {"Authorization": f"Bearer {token}"}
    
    response = requests.get(search_url, headers=headers)
    track_data = response.json()

    if "tracks" in track_data and track_data["tracks"]["items"]:
        track = track_data["tracks"]["items"][0]
        duration_ms = track["duration_ms"]  # Duration in milliseconds
        artist_id = track["artists"][0]["id"]
        return duration_ms, artist_id
    else:
        return None, None

# Fetch artist followers
def get_artist_followers(artist_id, token):
    if artist_id is None:
        return None  # Return None if artist ID is not found
    
    artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {"Authorization": f"Bearer {token}"}

    response = requests.get(artist_url, headers=headers)
    artist_data = response.json()

    return artist_data.get("followers", {}).get("total", None)

# Process DataFrame
def enrich_dataframe_with_spotify(df):
    token = get_access_token()

    durations = []
    followers = []

    for index, row in df.iterrows():
        song = row["Song"]
        artist = row["Artist"]

        duration_ms, artist_id = get_track_details(song, artist, token)
        artist_followers = get_artist_followers(artist_id, token)

        durations.append(duration_ms / 1000 if duration_ms else None)  # Convert to seconds
        followers.append(artist_followers)

        time.sleep(1)  # Avoid hitting API rate limits

    df["Duration (seconds)"] = durations
    df["Artist Followers"] = followers

    return df

# Example: Load your DataFrame


# Enrich DataFrame
df = enrich_dataframe_with_spotify(df_w_lyrics)
df = df.drop(df.columns[0], axis=1)
df = df.drop(df.columns[3], axis=1)



In [2]:
df_w_lyrics

NameError: name 'df_w_lyrics' is not defined

In [238]:
df

,Unnamed: 0,Song,Artist,lyrics,Lyrics,Duration (seconds),Artist Followers,Tempo,Danceability,Energy,Valence,Speechiness,Acousticness,Instrumentalness,Liveness
0,0.0,Ella Baila Sola,Eslabon Armado,"Compa, ¿qué le parece esa morra?\nLa que anda ...",NaN,165.671,6081474.0,None,None,None,None,None,None,None,None
1,1.0,These Words,Natasha Bedingfield,"My heart\nThese words are my own\n(Uh, yeah, u...",NaN,136.580,41112.0,None,None,None,None,None,None,None,None
2,2.0,"Guantanamera (feat. Ms. Lauryn Hill, Celia Cru...",Wyclef Jean,none,NaN,270.493,385395.0,None,None,None,None,None,None,None,None
3,3.0,Vanish Into You,Lady Gaga,Saw your face and mine\nIn a picture by our be...,NaN,244.915,35745647.0,None,None,None,None,None,None,None,None
4,4.0,Volver,Carmen Ferre,none,NaN,149.655,5988.0,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,NaN,Rescue,Lauren Daigle,You are not hidden\nThere's never been a momen...,None,215.613,2969606.0,None,None,None,None,None,None,None,None
1313,NaN,Wild,Beach House,My mother said to me that I would get in troub...,None,219.453,2540586.0,None,None,None,None,None,None,None,None
1314,NaN,You Came To Me,Beach House,Invite your sister\nInto the garden\nAll canno...,None,245.333,2540586.0,None,None,None,None,None,None,None,None
1315,NaN,Chariot,Beach House,Sunny day\nIn a chariot\nWere they waving back...,None,316.507,2540586.0,None,None,None,None,None,None,None,None


In [234]:
df

,Song,Artist,lyrics,Duration (seconds),Artist Followers
0,Ella Baila Sola,Eslabon Armado,"Compa, ¿qué le parece esa morra?\nLa que anda ...",165.671,6081474.0
1,These Words,Natasha Bedingfield,"My heart\nThese words are my own\n(Uh, yeah, u...",136.580,41112.0
2,"Guantanamera (feat. Ms. Lauryn Hill, Celia Cru...",Wyclef Jean,none,270.493,385395.0
3,Vanish Into You,Lady Gaga,Saw your face and mine\nIn a picture by our be...,244.915,35745647.0
4,Volver,Carmen Ferre,none,149.655,5988.0
...,...,...,...,...,...
1312,Rescue,Lauren Daigle,You are not hidden\nThere's never been a momen...,215.613,2969606.0
1313,Wild,Beach House,My mother said to me that I would get in troub...,219.453,2540586.0
1314,You Came To Me,Beach House,Invite your sister\nInto the garden\nAll canno...,245.333,2540586.0
1315,Chariot,Beach House,Sunny day\nIn a chariot\nWere they waving back...,316.507,2540586.0


## Add a langauge feature

In [248]:
df["language"] = [detect(lyric) for lyric in df["lyrics"]]

## Export data

In [1]:

df.to_csv('data.csv')

NameError: name 'df' is not defined